In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torch.utils.data import Dataset, DataLoader

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

torch.manual_seed(0)

# Any results you write to the current directory are saved as output.

/kaggle/input/fashionmnist/train-images-idx3-ubyte
/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte


In [42]:
class FashionMNISTDataset(Dataset):
    '''Fashion MNIST Dataset'''
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file
            transform (callable): Optional transform to apply to sample
        """
        
        data = pd.read_csv(csv_file);
        self.X = np.array(data.iloc[:, 1:])#.reshape(-1, 1, 28, 28)#.astype(float);
        self.Y = np.array(data.iloc[:, 0]);
        
        del data;
        self.transform = transform;
        
    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = self.X[idx];
        label = self.Y[idx];
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label);
    

In [43]:
from torchvision import datasets, transforms
# Define a transform to normalize the data
#transform = transforms.Compose([transforms.ToTensor(),
#                                transforms.Normalize((0.5,), (0.5,))])
train_dataset = FashionMNISTDataset(csv_file='/kaggle/input/fashionmnist/fashion-mnist_train.csv');
test_dataset = FashionMNISTDataset(csv_file='/kaggle/input/fashionmnist/fashion-mnist_test.csv')

In [44]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True);
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           shuffle=True);

In [45]:
# getstats
label_stats = np.zeros(10)
for images, labels in train_loader:
    bincnt = np.bincount(labels, minlength = 10)
    label_stats = np.add(label_stats, bincnt)
print("printing stats")
print(label_stats)
print("total samples read " + str(np.sum(label_stats)))

# reset loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True);

printing stats
[6000. 6000. 6000. 6000. 6000. 6000. 6000. 6000. 6000. 6000.]
total samples read 60000.0


In [46]:
## create NN
from torch import nn
import torch.nn.functional as F
from torch import optim

# TODO: Create the network, define the criterion and optimizer
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)


# TODO: Train the network here

epochs = 5

for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # Flatten MNIST images into a 784 long vector
    
        # TODO: Training pass
        optimizer.zero_grad()
        output = model(images.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_loader)}")
else:
    print("finished with training")
print("done")
        


Training loss: 0.8417775785999257
Training loss: 0.47115888074834716
Training loss: 0.4361429020825988
Training loss: 0.4323363372766133
Training loss: 0.4304101990102959
finished with training
done


In [47]:
import matplotlib.pyplot as plt
correct = 0
total_test = 0
for images, labels in test_loader:
    img = images[0].resize(1, 784).float()
    #plt.imshow(img.view(28, 28))
    with torch.no_grad():
        logps = model(img)
    ps = torch.exp(model(img))
    max_prob_idx = torch.argmax(ps)
    #print(max_prob_idx)
    #print(labels[0])
    if max_prob_idx.item() == labels[0].item():
        correct = correct + 1
    total_test = total_test + 1
print("Accuracy is " + str(float(correct/total_test) * 100))

Accuracy is 78.98089171974523
